<a href="https://colab.research.google.com/github/hoangtheanhhp/zalo_qa/blob/master/Question_Answering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
import sys
!rm -rf resources/ Framework.png Question\ Answering.ipynb reader.py relevance_ranking.py synonyms.py m\
gg_search.py entity_linking.py dataset/ bert-vietnamese-question-answering/ utils.py gg_search.py main.py requirements.txt

!rm -rf zalo_qa/

In [7]:
!rm -rf resources/

In [30]:
# Clone BERT repo and add bert in system path
!test -d zalo_qa || git clone https://github.com/hoangtheanhhp/zalo_qa
!mv /content/zalo_qa/*  /content/
!cp -r "/content/drive/My Drive/ZaloAI/resources" /content/

Cloning into 'zalo_qa'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 36 (delta 8), reused 36 (delta 8), pack-reused 0
Unpacking objects: 100% (36/36), done.


In [33]:
!pip install -r requirements.txt

In [1]:
from gg_search import GoogleSearch

from relevance_ranking import rel_ranking
from reader import Reader
reader = Reader()

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [4]:
ggsearch = GoogleSearch()

In [2]:
question = 'ai là người giàu nhất Việt Nam'

## Using google to find relevant documents

In [5]:
links, documents = ggsearch.search(question)
print(links, documents)
for i in range(0,min(len(documents),5)):
    print("Document", i)
    print("Link:", links[i])
    print("Sample content:\n", documents[i][:500],'.....')
    print('\n------------------------------\n')

['http://s.cafef.vn/top/ceo.chn', 'https://laodong.vn/doanh-nghiep-doanh-nhan/top-5-ti-phu-giau-nhat-viet-nam-dang-nam-giu-bao-nhieu-tien-785328.ldo', 'https://www.35express.org/top-10-nguoi-giau-nhat-viet-nam/', 'https://vi.wikipedia.org/wiki/Ph%E1%BA%A1m_Nh%E1%BA%ADt_V%C6%B0%E1%BB%A3ng', 'https://baomoi.com/gioi-sieu-giau-viet-nam-top-10-nguoi-giau-nhat-ho-la-nhung-ai/c/29894997.epi', 'https://dantri.com.vn/kinh-doanh/choang-ngop-tai-san-nguoi-giau-nhat-viet-nam-sap-cham-nguong-10-ty-usd-20190320131450330.htm', 'https://www.24h.com.vn/doanh-nhan/bat-ngo-voi-nhung-nguoi-giau-nhat-viet-nam-c606a1113890.html', 'http://tapchitaichinh.vn/tai-chinh-kinh-doanh/viet-nam-co-4-ty-phu-lot-top-ty-phu-the-gioi-2020-cua-forbes-321276.html', 'https://www.youtube.com/watch?v=ViHJhYYDRTw', 'http://vneconomy.vn/viet-nam-lan-dau-co-doanh-nhan-lot-danh-sach-200-nguoi-giau-nhat-the-gioi-20190216141641074.htm', 'https://vietnamnet.vn/vn/kinh-doanh/tai-chinh/top-3-giau-nhat-viet-nam-bat-ngo-trinh-van-quyet

## Find relevant passages from documents
- Extract keywords (must-have words) from question
- Filtering out documents that do not contain keywords from question
- Spliting documents into passages
- Filtering out passages that do not contain keywords from question
- Ranking remaining passages based on N-gram matching

In [6]:
passages = rel_ranking(question,documents)
for i in range(0, min(len(passages),5)):
    print('Passage ranked', i + 1,'\n')
    print(passages[i])
    print('\n------------------------------\n')

Passage ranked 1 

Theo Forbes, tài sản của tỷ phú Phạm Nhật Vượng đã tăng thêm 1,65 tỷ USD so với lần công bố gần nhất vào tháng 3. Với khối tài sản lên tới 8,25 tỷ USD, Chủ tịch Tập đoàn Vingroup tiếp tục là người giàu nhất Việt Nam và chính thức lọt top 200 người giàu nhất hành tinh (đứng thứ 195).

------------------------------

Passage ranked 2 

Trước đó vào năm 2010, Phạm Nhật Vượng là người giàu nhất trên TTCK Việt Nam với số tài sản gần 15.800 tỷ đồng, giàu thứ nhì Việt Nam (theo xếp hạng trên sàn chứng khoán) năm 2007, 2008.[4] Ông đạt được vị trí này vào năm 2007, khi Công ty Vinpearl thuộc nhóm các công ty của Vincom niêm yết 100 triệu cổ phiếu trên sàn giao dịch chứng khoán Tp. Hồ Chí Minh.

------------------------------

Passage ranked 3 

Ông Phạm Nhật Vượng tiếp tục được công nhận là người giàu nhất Việt Nam với tài sản định giá 6,6 tỷ USD, đứng thứ 239 thế giới, tăng 2,3 tỷ USD so với năm ngoái. Ông lần đầu tiên được Forbes vinh danh năm 2013, với 1,5 tỷ USD, đứng th

In [7]:
# Select top 40 paragraphs
passages = passages[:40]

### Using reading comprehend model (BERT) to extract answer for each passage

In [8]:
answers = reader.getPredictions(question,passages)

### Reranking passages by answer score

In [9]:
answers = [[passages[i], answers[i][0],answers[i][1]] for i in range(0,len(answers))]
answers = [a for a in answers if a[1] != '']
answers.sort(key = lambda x : x[2],reverse=True)

### Most relevant passages with potential answer

In [ ]:
for i in range(0,min(5,len(answers))):
    print("Passage: ", answers[i][0])
    print("Answer : ", answers[i][1])
    print("Score  : ", answers[i][2])
    print('\n')

Passage:  Ông Phạm Nhật Vượng tiếp tục được công nhận là người giàu nhất Việt Nam với tài sản định giá 6,6 tỷ USD, đứng thứ 239 thế giới, tăng 2,3 tỷ USD so với năm ngoái.
Answer :  Phạm Nhật Vượng
Score  :  0.9999549482428222


Passage:  VOV.VN - Ông Phạm Nhật Vượng là người giàu nhất Việt Nam với khối tài sản trị giá 6,7 tỷ USD, tương đương 152,9 ngàn tỷ đồng.
Answer :  Phạm Nhật Vượng
Score  :  0.9999439551387689


Passage:  Bảng xếp hạng người giàu nhất trên sàn chứng khoán

 Bảng xếp hạng người giàu nhất Việt Nam

 Bảng xếp hạng các tỷ phú chứng khoán

 Bảng xếp hạng các tỷ phú Việt Nam

 Ai là người giàu nhất Việt Nam

 Ai là người giàu nhất sàn chứng khoán

 Tỷ phú đô la của Việt Nam

 Trần Đình Long trở thành tỷ phú đô la

 Tài sản của ông Phạm Nhật Vượng

 Tài sản của Trịnh Văn Quyết

 Tài sản của bà Phương Thảo
Answer :  Trần Đình Long
Score  :  0.9999357839956173


Passage:  Cho đến nay vẫn chưa ai thay thế được vị trí người giàu nhất Việt Nam của ông Phạm Nhật Vượng. Giá tr

### Final answer

In [10]:
print("Final answer: ", answers[0][1])

Final answer:  Phạm Nhật Vượng
